# COCO dataset Filtering
- GPU 1개로 coco를 학습하려고 하면 너무 오래 걸림
- 데이터수가 너무 많기 때문에
- **특정 class만 가져와서 적은 데이터셋으로 training연습**
- **bbox의 사이즈가 1미만은 데이터는 제거**

In [1]:
import os

import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import CocoDetection

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np

%matplotlib inline

## Original

In [20]:
class myCocoDetection(CocoDetection):
    
    def __getitem__(self, index: int):
        idx = self.ids[index]
        image = self._load_image(idx)
        target = self._load_target(idx)
        
        bboxes, labels = [], []
        for obj in target:
            bboxes.append(obj['bbox'])
            labels.append(obj['category_id'])
            
        labels = torch.tensor(labels)
        if self.transform:
            image = self.transform(image)
            bboxes = self.transform(np.array(bboxes))
            
        return image, bboxes, labels
    
    def _load_image(self, idx: int):
        path = self.coco.loadImgs(idx)[0]["file_name"]
        return Image.open(os.path.join(self.root, path)).convert("RGB")
    
    def _load_target(self, idx):
        return self.coco.loadAnns(self.coco.getAnnIds(idx))
    
def collate_fn(batch):
    return tuple(zip(*batch))

In [21]:
test_path = '../../data/COCO/val2017'
test_ann = '../../data/COCO/annotations/instances_val2017.json'

test_set = myCocoDetection(root=test_path, annFile=test_ann, transform=transforms.ToTensor())

test_loader = DataLoader(test_set, batch_size=4, collate_fn=collate_fn)

images, bboxes, labels = iter(test_loader).next()

print('Shape of Image :', images[0].shape)
print('Shape of bboxes :', bboxes[0].shape)
print('Shape of labels :', len(labels))

loading annotations into memory...
Done (t=0.64s)
creating index...
index created!
Shape of Image : torch.Size([3, 426, 640])
Shape of bboxes : torch.Size([1, 20, 4])
Shape of labels : 4


### Number of class

In [22]:
len(test_set.coco.getCatIds())

80

## 1. bbox의 크기가 1보다 작은 데이터 제거
- img_ids : 각 image의 id
- ann_ids : 각 image에 있는 object의 id

In [26]:
from pycocotools.coco import COCO

In [41]:
test_path = '../../data/COCO/val2017'
test_ann = '../../data/COCO/annotations/instances_val2017.json'

coco = COCO(test_ann) # coco data의 target데이터
img_ids = list(sorted(coco.imgs.keys())) # 모든 이미지의 ids
ann_ids = coco.getAnnIds(img_ids[0])     # 이미지 내에 있는 object의 ids

print(len(img_ids))
print(len(ann_ids))

loading annotations into memory...
Done (t=0.84s)
creating index...
index created!
5000
20


In [ ]:
for img_id in ids:
    ann_i

In [23]:
class myCocoDetection(CocoDetection):
    
    def __getitem__(self, index: int):
        idx = self.ids[index]
        image = self._load_image(idx)
        target = self._load_target(idx)
        
        bboxes, labels = [], []
        for obj in target:
            bboxes.append(obj['bbox'])
            labels.append(obj['category_id'])
            
        labels = torch.tensor(labels)
        if self.transform:
            image = self.transform(image)
            bboxes = self.transform(np.array(bboxes))
            
        return image, bboxes, labels
    
    def _load_image(self, idx: int):
        path = self.coco.loadImgs(idx)[0]["file_name"]
        return Image.open(os.path.join(self.root, path)).convert("RGB")
    
    def _load_target(self, idx):
        return self.coco.loadAnns(self.coco.getAnnIds(idx))
    
def collate_fn(batch):
    return tuple(zip(*batch))